In [ ]:
pip install firebase_admin

In [ ]:
pip install psycopg2

In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore
import psycopg2

# Initialize Firebase Admin SDK
cred = credentials.Certificate('path_to_your_service_account_key.json')
try:
    firebase_admin.get_app()
except ValueError:
    firebase_admin.initialize_app(cred)

# Initialize Firestore
db = firestore.client()

# Connect to PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    database="your_database",
    user="your_username",
    password="your_password"
)
cur = conn.cursor()

In [ ]:
from datetime import datetime

def migrate_residents():
    docs = db.collection('clients').stream()

    for doc in docs:
        data = doc.to_dict()
        birthday = data.get('birthday').replace(tzinfo=None) if data.get('birthday') else None
        createdAt = data.get('createdAt').replace(tzinfo=None) if data.get('createdAt') else None
        lastVisit = data.get('lastVisit').replace(tzinfo=None) if data.get('lastVisit') else None
        updatedAt = data.get('updatedAt').replace(tzinfo=None) if data.get('updatedAt') else None

        try:
            cur.execute('''
                INSERT INTO residents (document_id, lastName, gender, updatedAt, lastNameLower, lastBackpack, lastSleepingBag, postalCode, middleInitial, createdAt, unhoused, isCheckedIn, lastVisit, BPCResident, idv1, notes, birthday, lastBusTicket, isBanned, firstNameLower, race, firstName, numKids, sheltered, id)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ''', (
                doc.id,
                data.get('lastName'),
                data.get('gender'),
                updatedAt,
                data.get('lastNameLower'),
                data.get('lastBackpack'),
                data.get('lastSleepingBag'),
                data.get('postalCode'),
                data.get('middleInitial'),
                createdAt,
                data.get('unhoused', False),
                data.get('isCheckedIn', False),
                lastVisit,
                data.get('BPCResident', False),
                data.get('idv1'),
                data.get('notes', ''),
                birthday,
                data.get('lastBusTicket'),
                data.get('isBanned', False),
                data.get('firstNameLower'),
                data.get('race', ''),
                data.get('firstName'),
                data.get('numKids', 0),
                data.get('sheltered', False),
                data.get('id')
            ))
            print(f"Inserted resident with document_id: {doc.id}")
        except Exception as e:
            print(f"Error inserting resident with document_id: {doc.id}, error: {e}")

migrate_residents()

In [ ]:
def migrate_visits():
    visits_docs = db.collection_group('visits').stream()

    for visit_doc in visits_docs:
        visit_data = visit_doc.to_dict()
        visit_createdAt = visit_data.get('createdAt').replace(tzinfo=None) if visit_data.get('createdAt') else None
        visit_updatedAt = visit_data.get('updatedAt').replace(tzinfo=None) if visit_data.get('updatedAt') else None
        parent_client_id = visit_doc.reference.parent.parent.id
        try:
            cur.execute('''
                INSERT INTO visits (resident_id, visit_id, clothingBoy, clothingGirl, sleepingBag, clothingWomen, notes, busTicket, giftCard, backpack, diaper, household, boyAge, girlAge, idv1, kidsQ, createdAt, financialAssistance, clothingMen, mensQ, womensQ, updatedAt, id, orcaCard)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ''', (
                parent_client_id,
                visit_doc.id,
                visit_data.get('clothingBoy', False),
                visit_data.get('clothingGirl', False),
                visit_data.get('sleepingBag', False),
                visit_data.get('clothingWomen', False),
                visit_data.get('notes', ''),
                visit_data.get('busTicket', 0),
                visit_data.get('giftCard', 0),
                visit_data.get('backpack', False),
                visit_data.get('diaper', 0),
                visit_data.get('household', ''),
                visit_data.get('boyAge', ''),
                visit_data.get('girlAge', ''),
                visit_data.get('idv1', ''),
                visit_data.get('kidsQ', 0),
                visit_createdAt,
                visit_data.get('financialAssistance', 0),
                visit_data.get('clothingMen', False),
                visit_data.get('mensQ', 0),
                visit_data.get('womensQ', 0),
                visit_updatedAt,
                visit_data.get('id', ''),
                visit_data.get('orcaCard', 0)
            ))
            print(f"Inserted visit for resident with document_id: {parent_client_id} and visit_id: {visit_doc.id}")
        except Exception as e:
            print(f"Error inserting visit for resident with document_id: {parent_client_id} and visit_id: {visit_doc.id}, error: {e}")

migrate_visits()

In [ ]:
def migrate_settings():
    settings_docs = db.collection('settings').stream()

    for settings_doc in settings_docs:
        settings_data = settings_doc.to_dict()
        settings_updatedAt = settings_data.get('updatedAt').replace(tzinfo=None) if settings_data.get('updatedAt') else None

        try:
            cur.execute('''
                INSERT INTO settings (document_id, backpackThreshold, daysEarlyThreshold, earlyOverride, id, reportingQueryLimit, sleepingBagThreshold, updatedAt)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            ''', (
                settings_doc.id,
                int(settings_data.get('backpackThreshold', 0)),
                int(settings_data.get('daysEarlyThreshold', 0)),
                settings_data.get('earlyOverride', False),
                settings_data.get('id', ''),
                int(settings_data.get('reportingQueryLimit', 0)),
                int(settings_data.get('sleepingBagThreshold', 0)),
                settings_updatedAt
            ))
            print(f"Inserted settings with document_id: {settings_doc.id}")
        except Exception as e:
            print(f"Error inserting settings with document_id: {settings_doc.id}, error: {e}")

migrate_settings()

In [ ]:
# Commit and close the connection
conn.commit()
cur.close()
conn.close()